## *Jeu de données IRVE*
-------------------
# Analyse des relations entre champs (intégrité des données)
------------------------------
## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- les principaux résultats sont regroupés sur un [tableau de bord](https://nbviewer.org/github/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/Analyse/IRVE_indicateurs.ipynb) 
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par l'identification d'incohérences de relations
- l'identification des incohérences permet de trouver des stratégies de réduction des écarts (dans l'exemple ci-dessous, on passe 36% d'écart à 3,8 %)  
- l'analyse des données permet de (re)construire le modèle de données qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 5 % des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Séparation doublons station - date de maj](#6---Séparation-doublons-station---date-de-maj)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d'erreurs résiduelles](#8---Exemples-d\'erreurs-résiduelles)

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve/Analyse)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-2024xxxx.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [1]:
from base64 import b64encode
from IPython.display import Image, display
with open('IRVE_modele.txt', 'r', encoding="utf-8") as f:
    modele = f.read()
display(Image(url="https://mermaid.ink/img/" + b64encode(modele.encode("ascii")).decode("ascii")))

------
# 2 - Initialisation
## initialisation logicielle

In [2]:
from datetime import datetime
import json
from tab_dataset import Cdataset
import pandas as pd
import ntv_pandas as npd
import pathlib

def analyse_integrite(data, schema, affiche=True, indic=True):
    '''analyse les relations du DataFrame 'data' définies dans le schéma 'schema'.
    Le nombre de ligne en erreur par relation (dict) est retourné et optionnellement affiché (paramètre 'affiche=True') . 
    Les lignes en erreur sont optionnellement ajoutées (paramètre 'indic=True') à 'data' sous forme de champs booléens par relation.
    '''
    dic_errors = Cdataset(data).check_relationship(schema)
    dic_count = {name: len(errors) for name, errors in dic_errors.items()}
    if affiche:
        for name, total in dic_count.items():
            print('{:<50} {:>5}'.format(name, total))
    if indic:
        data['ok'] = True
        for name, errors in dic_errors.items():
            data[name] = True
            data.loc[errors, name] = False
            data['ok'] = data['ok'] & data[name] 
    return dic_count

def add_nbre_pdc(data, affiche=True):
    '''ajoute un champ avec le nombre de pdc calculés et le compare au champ 'nbre_pdc'
    Le nombre d'écarts est affiché optionnellement (paramètre 'affiche=True').
    '''
    data['nb_pdc_calc'] = data.groupby('id_station_itinerance')['index'].transform('count')
    nbre_pdc = data['nbre_pdc'].astype('int')
    data['nbre_pdc_ok'] = data['nb_pdc_calc'] == nbre_pdc
    data['calc-nbre'] = data['nb_pdc_calc'] - nbre_pdc
    if affiche:
        ecart_nbre = len(data) - data['nbre_pdc_ok'].sum()
        print('{:<20} {:>5}'.format('nbre_pdc_ko', ecart_nbre))
        print(round(ecart_nbre / len(data) * 100), ' %')

## initialisation des données
- lecture du fichier issu de l'api

In [3]:
dates = ['2023-03-03', '2023-04-17', '2023-05-02', '2023-05-24', '2023-06-17', '2023-07-04', '2023-07-23', '2023-08-08',
         '2023-09-06', '2023-09-19', '2023-10-04', '2023-10-21', '2023-10-31', '2023-11-23', '2023-12-14', '2023-12-29',
         '2024-01-15', '2024-02-07', '2024-02-28', '2024-03-27', '2024-04-24', '2024-05-19', '2024-06-05']
date= dates[-1]
log = {'date_irve': date, 
    'file': 'consolidation-etalab-schema-irve-statique-v-2.3.1-'+date[:4]+date[5:7]+date[8:]+'.csv', # après 2024-02-06
    'chemin': str(pathlib.Path(npd.__file__).parent.parent.parent/"Environmental-Sensing"/"python"/"Validation"/"irve"/"Analyse")
      }
irve = pd.read_csv(log['chemin'] + '/' + log['file'], sep=',', low_memory=False, dtype='object')
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 

nombre de lignes :  77733


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [4]:
# complément à inclure dans le schéma de données
schema = {
    'relationships': [
         # relation unicité des pdl
         {"fields": ["id_pdc_itinerance", "index"],                    "link" : "coupled" },   
         # relations inter entités
         {"fields": ["id_station_itinerance", "contact_operateur"],    "link" : "derived" },
         {"fields": ["id_station_itinerance", "nom_enseigne"],         "link" : "derived" },
         {"fields": ["id_station_itinerance", "coordonneesXY"],        "link" : "derived" },
         {"fields": ["id_pdc_itinerance", "id_station_itinerance"],    "link" : "derived" },
         # relations intra entité - station
         {"fields": ["id_station_itinerance", "nom_station"],          "link" : "derived" },
         {"fields": ["id_station_itinerance", "implantation_station"], "link" : "derived" },
         #{"fields": ["id_station_itinerance", "date_maj"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "nbre_pdc"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "condition_acces"],      "link" : "derived" },
         {"fields": ["id_station_itinerance", "horaires"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "station_deux_roues"],   "link" : "derived" },
         # relations intra entité - localisation
         {"fields": ["coordonneesXY", "adresse_station"],              "link" : "derived" }
    ]
}
# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory = ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
             'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
             'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
             'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
             'date_maj', 'last_modified']
controles = ['index - id_pdc_itinerance', 'contact_operateur - id_station_itinerance', 'nom_enseigne - id_station_itinerance',
             'coordonneesXY - id_station_itinerance', 'id_station_itinerance - id_pdc_itinerance',
             'nom_station - id_station_itinerance', 'implantation_station - id_station_itinerance',
             'nbre_pdc - id_station_itinerance', 'condition_acces - id_station_itinerance', 'horaires - id_station_itinerance',
             'station_deux_roues - id_station_itinerance', 'adresse_station - coordonneesXY']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- de l'ordre de 3 % des points de charge sont hors itinerance

In [5]:
irve['siren_amenageur']

0        379629447
1        349009423
2        812328128
3        803719277
4        320342975
           ...    
77728    775682040
77729    552008914
77730    801177528
77731    902305556
77732    437854508
Name: siren_amenageur, Length: 77733, dtype: object

In [6]:
# séparation des données
data = irve
data[['id_station_itinerance','id_pdc_itinerance','restriction_gabarit']] = data[['id_station_itinerance','id_pdc_itinerance','restriction_gabarit']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'], 'soit : ', len(non_concerne)/(len(non_concerne)+len(itinerance)))
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  2909 soit :  0.037422973511893276
nombre de pdc en itinerance   :  74824


In [7]:
# arborescence des champs et nombre de valeurs différentes
print(itinerance_init.npd.analysis().tree())

-1: root-derived (74824)
   0 : index (0 - 74824)
   1 : contact_operateur (74667 - 157)
   2 : nom_enseigne (71662 - 3162)
   3 : coordonneesXY (56179 - 18645)
   4 : adresse_station (55671 - 19153)
   5 : id_station_itineranc (40622 - 34202)
   6 : nom_station (55479 - 19345)
   7 : implantation_station (74819 - 5)
   8 : nbre_pdc (74763 - 61)
   9 : condition_acces (74822 - 2)
   10: horaires (74312 - 512)
   11: station_deux_roues (74818 - 6)
   12: id_pdc_itinerance (2338 - 72486)
   13: date_maj (74124 - 700)
   14: last_modified (74305 - 519)


-----------------------
## 4 - Bilan initial intégrité
- 22 % des lignes présentent un défaut d'intégrité

In [8]:
# séparation données bonnes (itinerance_ok_1) et données résiduelles (itinerance_1)
res = analyse_integrite(itinerance_init, schema)
itinerance_ok_1 = itinerance_init.loc[itinerance_init.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance_init.loc[~itinerance_init.ok, relations].reset_index(drop=True)
itinerance_init = itinerance_init.loc[:, relations]
log['init_ok'] = len(itinerance_ok_1)
log['init_ko'] = len(itinerance_1)
print("\nnombre d'enregistrements sans erreurs : ", log['init_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['init_ko'])
print("taux d'erreur : ", round(log['init_ko'] / log['pdc_en_itinerance'] * 100), ' %')

index - id_pdc_itinerance                           4674
contact_operateur - id_station_itinerance             53
nom_enseigne - id_station_itinerance                 666
coordonneesXY - id_station_itinerance               1353
id_station_itinerance - id_pdc_itinerance           4034
nom_station - id_station_itinerance                 1358
implantation_station - id_station_itinerance         238
nbre_pdc - id_station_itinerance                    1488
condition_acces - id_station_itinerance               65
horaires - id_station_itinerance                      85
station_deux_roues - id_station_itinerance           789
adresse_station - coordonneesXY                     5862

nombre d'enregistrements sans erreurs :  63948
nombre d'enregistrements avec au moins une erreur :  10876
taux d'erreur :  15  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- 35% des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de réduire de 50% le nombre de lignes erronnées

In [9]:
# séparation doublons pdc (doublons_pdc) et données résiduelles (itinerance_2)
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by=['date_maj', 'last_modified']).duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
#doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
log['doublons_pdc'] = len(doublons_pdc)
log['sans_doublons_pdc'] = len(itinerance_2)
print('nombre de doublons pdc : ', log['doublons_pdc'], ' soit ', round(log['doublons_pdc']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_pdc'], ' soit ', round(log['sans_doublons_pdc']/log['init_ko'] * 100), ' %')

nombre de doublons pdc :  2338  soit  21  %
nombre de pdc sans doublon   :  8538  soit  79  %


In [10]:
# séparation données bonnes (itinerance_ok_3) et données résiduelles (itinerance_3)
res = analyse_integrite(itinerance_2, schema)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
log['etape3_ok'] = len(itinerance_ok_3)
log['etape3_ko'] = len(itinerance_3)
print("\nnombre d'enregistrements sans erreurs : ", log['etape3_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape3_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance             24
nom_enseigne - id_station_itinerance                  33
coordonneesXY - id_station_itinerance                711
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                 1322
implantation_station - id_station_itinerance         227
nbre_pdc - id_station_itinerance                     985
condition_acces - id_station_itinerance               38
horaires - id_station_itinerance                      70
station_deux_roues - id_station_itinerance           789
adresse_station - coordonneesXY                     4775

nombre d'enregistrements sans erreurs :  2281
nombre d'enregistrements avec au moins une erreur :  6257


-----------------------
## 6 - Séparation doublons station - date de maj
- la suppression des anciens pdc permet de réduire de 25% supplémentaire le nombre de lignes erronnées
- les dernières erreurs correspondent à des stations associées à 43 opérateurs et sont liées à des causes multiples

In [11]:
# séparation doublons stations (doublons_stat_maj) et données résiduelles (itinerance_4)
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
log['doublons_station'] = len(doublons_stat_maj)
log['sans_doublons_station'] = len(itinerance_4)
print('nombre de doublons stations : ', log['doublons_station'], ' soit ', 
      round(log['doublons_station']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_station'])

nombre de doublons stations :  1346  soit  12  %
nombre de pdc sans doublon   :  4911


In [12]:
# séparation données bonnes (itinerance_ok_5) et données résiduelles (itinerance_5 / itinerance_5_full)
res = analyse_integrite(itinerance_4, schema)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
log['etape5_ok'] = len(itinerance_ok_5)
log['etape5_ko'] = len(itinerance_5)
print("\nnombre d'enregistrements sans erreurs : ", log['etape5_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape5_ko'], ' soit ', 
      round(log['etape5_ko']/log['etape3_ko'] * 100), ' %')

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              2
nom_enseigne - id_station_itinerance                   5
coordonneesXY - id_station_itinerance                269
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  321
implantation_station - id_station_itinerance         199
nbre_pdc - id_station_itinerance                     374
condition_acces - id_station_itinerance               18
horaires - id_station_itinerance                      67
station_deux_roues - id_station_itinerance           448
adresse_station - coordonneesXY                     2345

nombre d'enregistrements sans erreurs :  1337
nombre d'enregistrements avec au moins une erreur :  3574  soit  57  %


In [13]:
# structure des données présentant une erreur et nombre de valeurs
print(itinerance_5.npd.analysis().tree())

-1: root-derived (3574)
   0 : index (0 - 3574)
   1 : contact_operateur (3524 - 50)
   2 : nom_enseigne (3381 - 193)
   3 : coordonneesXY (2877 - 697)
   4 : adresse_station (2416 - 1158)
   5 : id_station_itineranc (1789 - 1785)
      13: date_maj (1690 - 95)
   6 : nom_station (2512 - 1062)
   7 : implantation_station (3569 - 5)
   8 : nbre_pdc (3539 - 35)
   9 : condition_acces (3572 - 2)
   10: horaires (3492 - 82)
   11: station_deux_roues (3568 - 6)
   12: id_pdc_itinerance (0 - 3574)
   14: last_modified (3483 - 91)


In [14]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  4  erreurs :  11


In [15]:
# principaux opérateurs contributeur de défauts
operateurs = list(itinerance_5.drop_duplicates('contact_operateur')['contact_operateur'])
erreurs_op = [len(itinerance_5.loc[itinerance_5.contact_operateur == operateur]) for operateur in operateurs]
err_op_5, oper_5 = tuple(zip(*(sorted(zip(erreurs_op, operateurs), reverse=True))[:5]))

print(len(err_op_5), ' opérateurs représentent : ', round(sum(err_op_5) / sum(erreurs_op) * 100), ' % des écarts\n')
for err, op in zip(err_op_5, oper_5):
    print(err, op)

5  opérateurs représentent :  71  % des écarts

804 support@alizecharge.fr
584 supervision-ev.france@totalenergies.com
428 info@ionity.eu
408 assistance-technique@ouestcharge-paysdelaloire-moncompte.fr
322 sav@izivia.com


-----------------------
## 7 - Synthèse


### fichiers
Génération des fichiers intégrant les défauts d'intégrité :
- fichier csv des lignes résiduelles à traiter (IRVE_itinerance_residuel)
- fichier csv des données itinerance avec indicateur des données à corriger ou à ignorer (IRVE_itinerance_complet)
- fichier csv des données itinerance valides (IRVE_itinerance_valide)
- fichier csv des doublons (IRVE_itinerance_doublons)

In [16]:
# consolidation des données
itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
itinerance_doublons = itinerance[itinerance['doublons_a_supprimer']].reset_index(drop=True)
print('total des doublons à supprimer : ', len(itinerance_doublons))
itinerance_ok = itinerance[~itinerance['lignes_ko']].reset_index(drop=True)
print('nombre de pdc avec controles ok : ', len(itinerance_ok))

total des lignes à corriger :  3574
total des doublons à supprimer :  3684
nombre de pdc avec controles ok :  67566


In [17]:
# génération des fichiers
extension = log['date_irve'] +'.csv'
itinerance_5_full.to_csv('IRVE_itinerance_residuel' + extension)
itinerance.to_csv('IRVE_itinerance_complet' + extension)
itinerance_ok.to_csv('IRVE_itinerance_valide' + extension)
itinerance_doublons.to_csv('IRVE_itinerance_doublons' + extension)
log['IRVE_itinerance_residuel' + extension] = len(itinerance_5_full)
log['IRVE_itinerance_complet' + extension] = len(itinerance)
log['IRVE_itinerance_valide' + extension] = len(itinerance_ok)
log['IRVE_itinerance_valide_stat' + extension] = len(itinerance_ok.drop_duplicates('id_station_itinerance', keep='last'))
log['IRVE_itinerance_doublons' + extension] = len(itinerance_doublons)

In [18]:
l_prefix = list(pd.read_csv(log['chemin'] + '/' + 'Operations_Recharge_16-07-2024.csv', sep=';', low_memory=False, dtype='object')['Identifiant'])
l_prefix[0]

'FR073'

In [19]:
import numpy as np

df = pd.DataFrame({'a':["er", "rt", "", "ert"], 'b': [10, np.nan, 20, 1]})

df['b'] = df['b'].replace(np.nan, 0)
df['a'] = df['a'].replace("", "empty")
df
itinerance_ok['cable_t2_attache'].astype('bool')


0        True
1        True
2        True
3        True
4        True
         ... 
67561    True
67562    True
67563    True
67564    True
67565    True
Name: cable_t2_attache, Length: 67566, dtype: bool

In [20]:
from pydantic import (BaseModel, EmailStr, PastDate, PositiveFloat, PositiveInt, ValidationError)
from pydantic_extra_types.phone_numbers import PhoneNumber
from copy import copy
import re

PREFIX = list(pd.read_csv(log['chemin'] + '/' + 'Operations_Recharge_16-07-2024.csv', sep=';', low_memory=False, dtype='object')['Identifiant'])
    
NUMPDL = re.compile(r"^\d{14}$")

class FrenchPhoneNumber(PhoneNumber):
    """A phone number with french defaults."""
    default_region_code = "FR"
    
class EmailStr(BaseModel):
    x: EmailStr

class PastDate(BaseModel):
    x: PastDate

class PositiveInt(BaseModel):
    x: PositiveInt

class FrenchPhoneNumber(BaseModel):
    x: FrenchPhoneNumber

class NumPdl:
    def __init__(self, x=None):
        self.x = x
        if NUMPDL.fullmatch(x) is None:
            raise ValidationError

def check_value(value, clas=None, default=None):    
    try:
        clas(x=value)
    except:
        return default
    return value

def set_prefix(pref):    
    return pref if pref in PREFIX else PREFIX[0]
    
def pandas_to_qualicharge(data):
    '''
    convertit les champs suivants (et applique une valeur par défaut si erreur) :
        - contact_amenageur: EmailStr ('amenageur@email.fr')
        - contact_operateur: EmailStr ('example@email.fr')
        - telephone_operateur: FrenchPhoneNumber (06-06-06-06-06)
        - nbre_pdc: PositiveInt (0)
        - date_maj: PastDate ('1900-01-01')
        - num_pdl: "^\d{14}$" ('00001234567890')
        - date_mise_en_service: PastDate ('1900-01-01')
        - puissance_nominale: PositiveFloat (0.0)
    et applique le prefixe des stations aux pdc.
    '''
    quali = copy(data)

    quali['paiement_autre'] = quali['paiement_autre'].astype('bool')
    quali['cable_t2_attache'] = quali['cable_t2_attache'].astype('bool')
    quali['raccordement'] = quali['raccordement'].replace(np.nan, 'Direct')
    quali['siren_amenageur'] = quali['siren_amenageur'].astype('string').replace(["", np.nan], "000000000")
    quali['code_insee_commune'] = quali['code_insee_commune'].astype('string').replace(["", np.nan], "99999")
    
    quali['contact_amenageur'] = quali['contact_amenageur'].apply(check_value,  clas=EmailStr, default='amenageur@email.fr')
    quali['contact_operateur'] = quali['contact_operateur'].apply(check_value,  clas=EmailStr, default='operateur@email.fr')
    quali['telephone_operateur'] = quali['telephone_operateur'].apply(check_value,  clas=FrenchPhoneNumber, default='06-06-06-06-06')
    quali['nbre_pdc'] = quali['nbre_pdc'].apply(check_value,  clas=PositiveInt, default=0)
    quali['date_maj'] = quali['date_maj'].apply(check_value,  clas=PastDate, default='1900-01-01')
    quali['num_pdl'] = quali['num_pdl'].apply(check_value,  clas=NumPdl, default='00001234567890')
    quali['date_mise_en_service'] = quali['date_mise_en_service'].apply(check_value,  clas=PastDate, default='1900-01-01')
    quali['puissance_nominale'] = quali['puissance_nominale'].apply(check_value,  clas=PositiveFloat, default=0.01)
    prefix = quali['id_station_itinerance'].astype('str').str.slice(stop=5).apply(set_prefix)
    quali['id_station_itinerance'] = prefix.str.cat(quali['id_station_itinerance'].astype('str').str.slice(start=5))
    quali['id_pdc_itinerance'] = prefix.str.cat(quali['id_pdc_itinerance'].astype('str').str.slice(start=5))

    quali['restriction_gabarit'] = quali['restriction_gabarit'].astype('string')
    return quali

itinerance_quali = pandas_to_qualicharge(itinerance_ok)

In [21]:
type(itinerance_quali.loc[85, 'restriction_gabarit'])
itinerance_quali['paiement_autre'].unique()

array([ True])

In [22]:
itinerance_quali.loc[:80, relations]


,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,0,oscar@electromaps.com,Hotel saint alban,"[3.407609123225763, 43.41959147913006]","31 Avenue d'Agde 34120 Nézignan-l'Évêque, Hérault",FR073011062174,Hotel saint alban,Parking privé à usage public,1,Accès libre,"Mo-Fr 08:00-12:00,Mo-Fr 14:00-18:00,Th 08:00-1...",FALSE,FR073011062174,2022-04-25,2024-01-19T07:47:00.381000+00:00
1,1,oscar@electromaps.com,Hôtel Restaurant Campanile Nogent-sur-Marne,"[2.493569567590577, 48.832677935169805]",13 Quai du Port 94130 Nogent-sur-Marne France,FR073012292701,Hôtel Restaurant Campanile Nogent-sur-Marne,Parking privé à usage public,2,Accès libre,"Mo-Fr 08:00-12:00,Mo-Fr 14:00-18:00,Th 08:00-1...",FALSE,FR073012292701,2022-05-12,2024-01-19T07:47:00.381000+00:00
2,2,oscar@electromaps.com,Résidence les calanques,"[8.657888301758055, 41.908579309038004]",52 Rte des Sanguinaires 20000 Ajaccio,FR073012308585,Résidence les calanques,Parking privé à usage public,8,Accès libre,"Mo-Fr 08:00-12:00,Mo-Fr 14:00-18:00,Th 08:00-1...",FALSE,FR073012308585,2022-04-19,2024-01-19T07:47:00.381000+00:00
3,3,oscar@electromaps.com,1PACTE,"[5.476711409891, 43.476583984941]","Chem. de Chabanu, 13120 Gardanne",FR073012616553,1PACTE,Parking privé à usage public,2,Accès libre,"Mo-Fr 08:00-12:00,Mo-Fr 14:00-18:00,Th 08:00-1...",FALSE,FR073012616553,2022-05-04,2024-01-19T07:47:00.381000+00:00
4,4,ismael@electromaps.com,SAS La Tuiliere,"[5.143766265497639, 43.3292004491334]","34, avenue Draïo de la Mar 13620 Carry-le-Rouet",FR073028067822,Carry-le-Rouet,Parking privé à usage public,2,Accès libre,24/7,false,FR073028067822,2023-06-06,2024-01-19T07:48:04.429000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,109,contact@nexteneo.com,Ville d'Annecy,"[6.116686, 45.917568]",[33 Boulevard du Fier][74000][Annecy],FR0NXPOOLC4BHY,Ville d'Annecy - Parc des sports,Parking public,9,Accès réservé,24/7,FALSE,FR0NXEVSEXALXR,2024-06-08,2024-05-30T10:20:40.802000+00:00
77,110,contact@nexteneo.com,SA TRANSPORTS FRANCONY,"[6.111914, 45.881672]",[10 Rue de la Bouverie][74600][Seynod],FR0NXPOOLTMC8P,SABA,Parking privé réservé à la clientèle,3,Accès réservé,Mo-Fr 12:00-14:00,FALSE,FR0NXEVSEXB9YG,2023-06-01,2024-05-30T10:20:40.802000+00:00
78,111,contact@nexteneo.com,CLUB MED,"[6.897273, 45.459233]",[Rue de la Legettaz][73150][Val-d'Isère],FR0NXPOOLNXWJ8,CLUB MED,Parking privé réservé à la clientèle,20,Accès réservé,24/7,FALSE,FR0NXEVSEYAT9J,2023-02-10,2024-05-30T10:20:40.802000+00:00
79,112,contact@nexteneo.com,CLUB MED,"[6.897273, 45.459233]",[Rue de la Legettaz][73150][Val-d'Isère],FR0NXPOOLNXWJ8,CLUB MED,Parking privé réservé à la clientèle,20,Accès réservé,24/7,FALSE,FR0NXEVSEYGA88,2023-02-10,2024-05-30T10:20:40.802000+00:00


In [23]:
itinerance_quali.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,0,oscar@electromaps.com,Hotel saint alban,"[3.407609123225763, 43.41959147913006]","31 Avenue d'Agde 34120 Nézignan-l'Évêque, Hérault",FR073011062174,Hotel saint alban,Parking privé à usage public,1,Accès libre,"Mo-Fr 08:00-12:00,Mo-Fr 14:00-18:00,Th 08:00-1...",FALSE,FR073011062174,2022-04-25,2024-01-19T07:47:00.381000+00:00
1,1,oscar@electromaps.com,Hôtel Restaurant Campanile Nogent-sur-Marne,"[2.493569567590577, 48.832677935169805]",13 Quai du Port 94130 Nogent-sur-Marne France,FR073012292701,Hôtel Restaurant Campanile Nogent-sur-Marne,Parking privé à usage public,2,Accès libre,"Mo-Fr 08:00-12:00,Mo-Fr 14:00-18:00,Th 08:00-1...",FALSE,FR073012292701,2022-05-12,2024-01-19T07:47:00.381000+00:00
2,2,oscar@electromaps.com,Résidence les calanques,"[8.657888301758055, 41.908579309038004]",52 Rte des Sanguinaires 20000 Ajaccio,FR073012308585,Résidence les calanques,Parking privé à usage public,8,Accès libre,"Mo-Fr 08:00-12:00,Mo-Fr 14:00-18:00,Th 08:00-1...",FALSE,FR073012308585,2022-04-19,2024-01-19T07:47:00.381000+00:00
3,3,oscar@electromaps.com,1PACTE,"[5.476711409891, 43.476583984941]","Chem. de Chabanu, 13120 Gardanne",FR073012616553,1PACTE,Parking privé à usage public,2,Accès libre,"Mo-Fr 08:00-12:00,Mo-Fr 14:00-18:00,Th 08:00-1...",FALSE,FR073012616553,2022-05-04,2024-01-19T07:47:00.381000+00:00
4,4,ismael@electromaps.com,SAS La Tuiliere,"[5.143766265497639, 43.3292004491334]","34, avenue Draïo de la Mar 13620 Carry-le-Rouet",FR073028067822,Carry-le-Rouet,Parking privé à usage public,2,Accès libre,24/7,false,FR073028067822,2023-06-06,2024-01-19T07:48:04.429000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67561,77618,advenir@zeborne.com,TOYOTA NICE - NOVELLIPSE,"[7.201547, 43.6885902]",108 Avenue Simone Veil 06200 Nice,FRZTLE24DC55361,Toyota - Nice - 24kW DC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:30,Mo-Fr 13:30–19:00,Sat 09:00-...",false,FRZTLE24DC55361,2021-02-24,2024-01-19T07:46:20.729000+00:00
67562,77619,advenir@zeborne.com,GARAGE NELLO CHELI - BEAUNE,"[4.86082418, 47.0236522]",RUE LUCIEN PERRIAUX ZI DE VIGNOLES 21200 Beaune,FRZTLE24DC61318,Toyota - Beaune - 24kW DC,Parking public,1,Accès libre,"Mo-Fr 07:45-12:00,Mo-Fr 13:45-19:00,Sat 09:00-...",false,FRZTLE24DC61318,2021-05-24,2024-01-19T07:46:19.511000+00:00
67563,77620,advenir@zeborne.com,GARAGE NELLO CHELI - BEAUNE,"[4.86082418, 47.0236522]",RUE LUCIEN PERRIAUX ZI DE VIGNOLES 21200 Beaune,FRZTLE24DC61319,Toyota - Beaune - 24kW DC,Parking public,1,Accès libre,"Mo-Fr 07:45-12:00,Mo-Fr 13:45-19:00,Sat 09:00-...",false,FRZTLE24DC61319,2021-05-24,2024-01-19T07:46:19.511000+00:00
67564,77621,advenir@zeborne.com,TOYOTA ANNEMASSE - DEGENEVE,"[6.2174783, 46.1858677]",8 RUE DE L'INDUSTRIE 74240 Gaillard,FRZTLE24DC63446,Toyota - Annemasse - 24kW DC,Parking public,1,Accès libre,"Mo-Fr 08:00-12:00,Mo-Fr 14:00-19:00,Sat 09:00-...",false,FRZTLE24DC63446,2021-06-16,2024-01-19T07:46:18.987000+00:00


In [24]:
itinerance_quali['paiement_autre'].to_csv('test.csv', index=False, quoting=csv.QUOTE_ALL, na_rep='empty')

NameError: name 'csv' is not defined

In [ ]:
import csv
itinerance_quali.to_csv('IRVE_itinerance_valide_quali' + extension, index=False, quoting=csv.QUOTE_ALL, na_rep='empty')

In [ ]:
for op in oper_5:
    print('IRVE_itinerance_residuel' + '_' + op + '_' + extension)
    itinerance_5_full.loc[itinerance_5_full.contact_operateur == op].to_csv('IRVE_itinerance_residuel' + '_' + op + '_' + extension)

### vérification de l'intégrité

In [ ]:
# vérification de l'absence d'erreurs
res = analyse_integrite(itinerance_ok.loc[:, relations], schema, affiche=False)
log['bilan_erreurs'] = sum(res.values())
log['date'] = datetime.now().isoformat()
print('bilan intégrité :')
print('    erreurs : ', log['bilan_erreurs'])

In [ ]:
# structure des données bonnes
#print(Cdataset(itinerance_ok.loc[:, mandatory]).tree())
print(itinerance_ok.loc[:, mandatory].npd.analysis().tree())

### Indicateurs

In [ ]:
# génération des indicateurs liés aux règles
regles = ['Pdc non unique', 'Station multi-operateurs', 'Station multi-enseignes', 'Station multi-localisations', 
          'Pdc multi-stations', 'station avec plusieurs noms', 'station multi-implantations', 
          'nombre de pdc par station incoherent', 'station multi-acces', 'station multi-horaires', 
          'acces deux-roues incoherent', 'localisation multi-adresses']
principal = [16, 17, 18, 19, 20]
secondaire = [21, 22, 23, 24, 25, 26, 27]
irve = itinerance_5_full # residuel
total = len(irve)
indic = {}

irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']
indic['principal pdc'] = int(total - irve['principal'].sum())
indic['secondaire pdc'] = int(total - irve['secondaire'].sum())

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
indic['principal stat'] = len(irve_p)
indic['secondaire stat'] = len(irve_s)

In [ ]:
# ajout des principaux opérateurs contributeur de défauts
indic['operateurs'] = list(oper_5)
indic['erreurs_operateurs'] = list(err_op_5)

In [ ]:
# ajout de l'indicateur d'écart entre nombre de pdc calculés et saisis
add_nbre_pdc(irve, affiche=True)
indic['nb_pdc_calc-nbre_pdc'] = int(irve['calc-nbre'].sum())

In [ ]:
# indicateur d'écart entre nombre de pdc calculés et saisis sur les données sans doublon
itinerance_sans_doublons = itinerance[~itinerance['doublons_a_supprimer']].reset_index(drop=True)
add_nbre_pdc(itinerance_sans_doublons, affiche=True)

In [ ]:
len(itinerance_4)

In [ ]:
# stockage des indicateurs
log |= indic
log_file = log['chemin'] + '/' + 'logfile.txt'
#with open('logfile.txt', 'a', encoding="utf-8") as f:
with open(log_file, 'a', encoding="utf-8") as f:
    f.write(json.dumps(log) + '\n')

In [ ]:
itinerance_5_full.columns[16:28]

-----------------------
## 8 - Exemples d'erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (3) (109 pdc)
- exemple station 'FR55CP92140' avec 29 pdc ! : 5 coordonnées XY, 5 nom_station, 2 nbre_pdc)

In [ ]:
maxi.loc[:, relations]

In [ ]:
itinerance_4.loc[itinerance_4.id_station_itinerance == 'FR55CP92140', relations]

### Repartition des defauts par operateur
Répartition pour les 5 opérateurs présentant le plus de défauts

In [ ]:
print('PdC avec une incohérence : ')
for operateur in oper_5:
    print('\n', sum(itinerance_5.contact_operateur == operateur))
    print('Répartition des incohérences ', operateur, ' :')
    for cont in controles:
        #print(cont, sum((itinerance_4['contact_operateur'] == 'supervision-ev.france@totalenergies.com') & ~(itinerance_4[cont])))
        print('{:<50} {:>5}'.format(cont, sum((itinerance_4['contact_operateur'] == operateur) & ~(itinerance_4[cont]))))

### Defauts operateur Ouestcharge-Paysdelaloire
Les défauts identifiés concernent des stations utilisant un même identifiant. Par exemple, la station 'FRADPP91479018' contient 82 points de recharge avec plusieurs adresses et un nombre de points de rechage variable. Cet exemple concerne des stations implantées sur plusieurs étages d'un parking (on devrait donc a minima identifier une station par étage).

In [ ]:
sum(itinerance_sans_doublons.contact_operateur == 'assistance-technique@ouestcharge-paysdelaloire-moncompte.fr')

In [ ]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'assistance-technique@ouestcharge-paysdelaloire-moncompte.fr', relations]

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-1.397512, 47.074964]', relations]

### Defauts operateur ALIZECHARGE
Les défauts identifiés concernent des stations utilisant un même identifiant. Par exemple, la station 'FRADPP91479018' contient 82 points de recharge avec plusieurs adresses et un nombre de points de rechage variable. Cet exemple concerne des stations implantées sur plusieurs étages d'un parking (on devrait donc a minima identifier une station par étage).

In [ ]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'support@alizecharge.fr', relations][50:200]

In [ ]:
# example [2.362554, 48.728787]
adp = itinerance_4.loc[itinerance_4.coordonneesXY == '[2.362554, 48.728787]']
adresses = list(adp.drop_duplicates('adresse_station')['adresse_station'])
print(adresses)

print(adp.loc[~adp['nom_station - id_station_itinerance'], ['nom_station', 'id_station_itinerance']])

print(adp.loc[~adp['implantation_station - id_station_itinerance'], ['implantation_station', 'id_station_itinerance']])

print(adp.loc[~adp['nbre_pdc - id_station_itinerance'], ['nbre_pdc', 'id_station_itinerance']])

itinerance_4.loc[itinerance_4.coordonneesXY == '[2.362554, 48.728787]', relations]

In [ ]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'support@alizecharge.fr', relations].loc[~itinerance_4['adresse_station - coordonneesXY']][500:600]

In [ ]:
itinerance_4.loc[itinerance_4.id_station_itinerance == 'FRADPE914790181', relations]

In [ ]:
itinerance_4.loc[itinerance_4.id_station_itinerance == 'FRADPE914790181', controles]

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[2.362554, 48.728787]', controles]

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-1.666373, 47.527509]', relations]

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-1.022143, 46.877453]', relations]

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[2.362554, 48.728787]', relations]

### Defauts operateur IZIVIA
Les défauts au nombre de 280 sont liés à une incohérence entre adresse et coordonnées (une même coordonnée a plusieurs adresses différentes). Ce défaut est lié soit à des erreurs de saisie, soit à des ajouts (ou remplacements) ultérieurs de stations à une même localisation.

Les stations Izivia en erreur ont un id_station_itinerance identique à l'id_pdc_itinerance, ce qui se traduit par des adresses multiples pour une même coordonnée.

Par exemple, sur un parking [-0.530598, 47.398424], on trouve 17 stations avec la même coordonnée, le même identifiant entre point de recharge et station et un nombre de PdC indiqué entre 1 et 3.

In [ ]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'sav@izivia.com', relations]

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-0.530598, 47.398424]', relations]

### Defauts operateur IONITY
Les défauts (480 points de recharge) sont liés à une incohérence entre adresse et coordonnées (une même coordonnée a plusieurs adresses différentes). Ce défaut est lié soit à des erreurs de saisie, soit à des ajouts (ou remplacements) ultérieurs de stations à une même localisation.

Les stations Ionity en erreur ont un id_station_itinerance identique à l'id_pdc_itinerance ainsi qu'un ancien pdc de regroupement par station avec un nom d'enseigne différent (IONITY GMBH), ce qui se traduit par des adresses multiples pour une même coordonnée.

Par exemple, sur l'aire de Mornas [4.732409, 44.194832], on trouve 26 stations avec la même coordonnée, le même identifiant entre point de recharge et station et un nombre de PdC indiqué de 23 pour chaque station.

In [ ]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'info@ionity.eu', relations]

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[1.580008, 48.468782]', relations]

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[4.732409, 44.194832]', relations]

In [ ]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'info@ionity.eu', relations]

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[0.366838, 46.701436]', relations]

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[2.523685, 48.9908]', relations]

### Cohérence implantation_station - id_station
- 107 pdc sont liés à une erreur de choix d'implantation (couplé à d'autres erreurs)

In [ ]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

### Cohérence nom_station - id_station
- 210 pdc sont associés à une station avec un nom non cohérent

In [ ]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

### Coherence adresse - coordonnees
- 2337 pdc ont une adresse non cohérente avec les coordonnées géographiques

In [ ]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[5.2592028, 46.1746523]', relations]

In [ ]:
itinerance_4.loc[itinerance_4.adresse_station == 'RN 75 ZI DE NOIREFONTAINE 01250 Montagnat', relations]

In [ ]:
itinerance_4.loc[itinerance_4.adresse_station == ' RN 75 ZI DE NOIREFONTAINE 01250 Montagnat', relations]

### Cohérence station - coordonnées
- 218 pdc sont associés à des stations avec plusieurs coordonnées (majorité AlizeCharge)

In [ ]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations]

In [ ]:
itinerance_5.loc[itinerance_5.id_station_itinerance	 == 'FRTLSE31555040', relations]

In [ ]:
itinerance_5.loc[itinerance_5.id_station_itinerance	 == 'FRFR1PEHNKRRPQNR', relations]

- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [ ]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

- exemple : une station avec plusieurs coordonnées -> regroupement de stations ?

In [ ]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR55CPBP514', relations]

- exemple : plusieurs stations d'opérateurs différents avec les mêmes coordonnées -> suppression non effectuée lors d'un changement d'opérateur ?

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[0.08745032, 48.40887156]', relations]

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-4.374469, 48.471878]', relations]

- exemple : quatre stations avec une localisation identique mais deux adresses différentes + id identique entre station et pdc -> erreur d'adresse et erreur d'id_station ?

In [ ]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-0.366184, 43.332508]', relations]

### Incoherence nbre_pdc
- 583 pdc ont un champ 'nbre_pdc' mal documenté

In [ ]:
itinerance_4.loc[~itinerance_4['nbre_pdc - id_station_itinerance'], relations][200:215]

### coherence station - enseigne
- 5 stations ont plusieurs noms d'enseigne

In [ ]:
itinerance_4.loc[~itinerance_4['nom_enseigne - id_station_itinerance'], relations]

In [ ]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRFR1EAYRU', relations]

In [ ]:
itinerance_init.loc[itinerance_init.nom_enseigne == 'RME Loos (FR-FR1)', relations]

### cohérence station - horaires
- exemple de station avec plusieurs types d'horaire (15)

In [ ]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS23D2302001', relations]

### cohérence station - deux-roues
- exemple de station avec plusieurs indications d'accessibilité deux-roues (583)

In [ ]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRV75PPX1716', relations]